In [1]:
import os
import subprocess
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

import shutil
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

pd.set_option("max_colwidth", 100)
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

from google.cloud import storage

In [3]:
# ! pip install pycountry
# ! pip install pycountry_convert

import pycountry as pyc
import pycountry_convert as pyc_c

     |████████████████████████████████| 10.1 MB 5.3 MB/s eta 0:00:01
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746865 sha256=f4a761e31ad26c3164e524375cce3ba665e5689551116e6fe415be3d1371ec4e
  Stored in directory: /root/.cache/pip/wheels/09/eb/0d/4ee773c6a4aadc2a43cb5c1d07f268f13c4cdc0eec88e7c1ef
Successfully built pycountry
     |████████████████████████████████| 280 kB 8.1 MB/s eta 0:00:01
     |████████████████████████████████| 217 kB 59.5 MB/s eta 0:00:01
     |████████████████████████████████| 98 kB 12.5 MB/s eta 0:00:01


In [4]:
%%time

path = 'gs://msca-bdp-students-bucket/shared_data/kaihayden/final1'

tweets_df = spark.read.parquet(path)

CPU times: user 1.77 ms, sys: 3.69 ms, total: 5.46 ms
Wall time: 6.54 s


In [5]:
# tweets_df.columns

In [5]:
tweets_original = tweets_df.filter(col('is_original')==1)

In [6]:
# tweets_original.count()

In [ ]:
# tweets_original.select([count(when(col(c).isNull(), c)).alias(c) for c in tweets_original.columns]).show(vertical=True)

-RECORD 0--------------------
 created_at        | 0       
 dt_date           | 0       
 dt_datehour       | 0       
 dt_day            | 0       
 dt_hour           | 0       
 dt_month          | 0       
 dt_year           | 0       
 ext_qt_user_id    | 2739456 
 ext_rp_user_id    | 2648070 
 ext_rt_id         | 2739456 
 ext_rt_user_id    | 2739456 
 is_original       | 0       
 reply_count       | 0       
 twt_country       | 2692688 
 twt_hashtags      | 0       
 twt_id            | 0       
 twt_importance    | 0       
 twt_likes         | 0       
 twt_location      | 2692688 
 twt_location_full | 2692688 
 twt_location_type | 2692688 
 twt_quote_count   | 0       
 twt_reply_count   | 0       
 twt_retwt_count   | 0       
 twt_text          | 0       
 usr_desc          | 278311  
 usr_followers     | 0       
 usr_id            | 0       
 usr_location      | 739986  
 usr_name          | 0       
 usr_tweet_count   | 0       
 usr_verified      | 0       



In [33]:
tweets_df.select([count(when(col(c).isNull(), c)).alias(c) for c in tweets_original.columns]).show(vertical=True)

-RECORD 0---------------------
 created_at        | 0        
 dt_date           | 0        
 dt_datehour       | 0        
 dt_day            | 0        
 dt_hour           | 0        
 dt_month          | 0        
 dt_year           | 0        
 ext_qt_user_id    | 19016605 
 ext_rp_user_id    | 20700808 
 ext_rt_id         | 7350818  
 ext_rt_user_id    | 7350818  
 is_original       | 0        
 reply_count       | 0        
 twt_country       | 23779768 
 twt_hashtags      | 0        
 twt_id            | 0        
 twt_importance    | 0        
 twt_likes         | 0        
 twt_location      | 23779768 
 twt_location_full | 23779768 
 twt_location_type | 23779768 
 twt_quote_count   | 0        
 twt_reply_count   | 0        
 twt_retwt_count   | 0        
 twt_text          | 0        
 usr_desc          | 4470803  
 usr_followers     | 0        
 usr_id            | 0        
 usr_location      | 9017922  
 usr_name          | 0        
 usr_tweet_count   | 0        
 usr_ver

### only about 47000 original tweets with locations, we'll try to extract continents from locations

In [8]:
country_name = [i.name for i in pyc.countries]
country_3 = [i.alpha_3 for i in pyc.countries]
country_2 = [i.alpha_2 for i in pyc.countries]

In [9]:
def cont_name(a):
    try:
        return pyc_c.country_alpha2_to_continent_code(pyc_c.country_name_to_country_alpha2(a))
    except:
        return None
def cont_3(a):
    try:
        return pyc_c.country_alpha2_to_continent_code(pyc_c.country_alpha3_to_country_alpha2(a))
    except:
        return None
def cont_2(a):
    try:
        return pyc_c.country_alpha2_to_continent_code(a)
    except:
        return None

# 'NA','AS','AF','EU','SA','OC'

In [10]:
states = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
states = [', '+ s for s in states]

na_full = [country for country in country_name if cont_name(country) == 'NA']
na_full.extend(['America','Curacao','Saint Martin','Virgin Islands'])
as_full = [country for country in country_name if cont_name(country) == 'AS']
as_full.extend(['Taiwan','Hong Kong','Vietnam','Korea','Laos','Macau','Iran'])
af_full = [country for country in country_name if cont_name(country) == 'AF']
eu_full = [country for country in country_name if cont_name(country) == 'EU']
eu_full.extend(['England','Scotland','Ireland','Wales'])
sa_full = [country for country in country_name if cont_name(country) == 'SA']
sa_full.extend(['Venezuela','Bolivia','Falkland'])
oc_full = [country for country in country_name if cont_name(country) == 'OC']
oc_full.extend(['Micronesia'])

na_full1 = [', '+ c for c in na_full]
as_full1 = [', '+ c for c in as_full]
af_full1 = [', '+ c for c in af_full]
eu_full1 = [', '+ c for c in eu_full]
sa_full1 = [', '+ c for c in sa_full]
oc_full1 = [', '+ c for c in oc_full]

na_3 = [country for country in country_3 if cont_3(country) == 'NA']
na_3.extend(states)
as_3 = [country for country in country_3 if cont_3(country) == 'AS']
af_3 = [country for country in country_3 if cont_3(country) == 'AF']
eu_3 = [country for country in country_3 if cont_3(country) == 'EU']
eu_3.extend(['ENG','SCO','WAL','IRE'])
sa_3 = [country for country in country_3 if cont_3(country) == 'SA']
oc_3 = [country for country in country_3 if cont_3(country) == 'OC']

na_2 = [country for country in country_2 if cont_2(country) == 'NA']
na_2.remove('DM')
as_2 = [country for country in country_2 if cont_2(country) == 'AS']
as_2.remove('IN')
as_2.remove('LA')
af_2 = [country for country in country_2 if cont_2(country) == 'AF']
eu_2 = [country for country in country_2 if cont_2(country) == 'EU']
eu_2.extend(['UK'])
sa_2 = [country for country in country_2 if cont_2(country) == 'SA']
oc_2 = [country for country in country_2 if cont_2(country) == 'OC']

In [11]:
tweets_usr_locations = tweets_df.select(["usr_id","usr_location"]).na.drop().dropDuplicates()

In [12]:
# tweets_usr_locations.show(5)

In [13]:
tweets_na = tweets_usr_locations.withColumn('loc0', when(lower(col("usr_location")).rlike("|".join(["(" + kw.lower() + ")" for kw in na_full1])),lit(4)).otherwise(lit(0)))\
                                .withColumn('loc1', when(col("usr_location").rlike("|".join(["(" + kw + ")" for kw in na_3])),lit(4)).otherwise(lit(0)))\
                                .withColumn('loc2', when(lower(col("usr_location")).rlike("|".join(["(" + kw.lower() + ")" for kw in na_full])),lit(3)).otherwise(lit(0)))\
                                .withColumn('loc3', when(col("usr_location").rlike("|".join(["(" + kw + ")" for kw in na_2])),lit(2)).otherwise(lit(0)))\
                                .withColumn('continent',lit('NA'))
    
tweets_na = tweets_na.withColumn("score", col('loc0')+col('loc1')+col('loc2')+col('loc3'))

tweets_na = tweets_na.select('usr_id','usr_location','score','continent')\
                     .filter(col('score').isNotNull()).orderBy('score',ascending=False)

In [14]:
tweets_as = tweets_usr_locations.withColumn('loc0', when(lower(col("usr_location")).rlike("|".join(["(" + kw.lower() + ")" for kw in as_full1])),lit(4)).otherwise(lit(0)))\
                                .withColumn('loc1', when(col("usr_location").rlike("|".join(["(" + kw + ")" for kw in as_3])),lit(4)).otherwise(lit(0)))\
                                .withColumn('loc2', when(lower(col("usr_location")).rlike("|".join(["(" + kw.lower() + ")" for kw in as_full])),lit(3)).otherwise(lit(0)))\
                                .withColumn('loc3', when(col("usr_location").rlike("|".join(["(" + kw + ")" for kw in as_2])),lit(2)).otherwise(lit(0)))\
                                .withColumn('continent',lit('AS'))
    
tweets_as = tweets_as.withColumn("score", col('loc0')+col('loc1')+col('loc2')+col('loc3'))

tweets_as = tweets_as.select('usr_id','usr_location','score','continent')\
                     .filter(col('score').isNotNull()).orderBy('score',ascending=False)

In [15]:
tweets_af = tweets_usr_locations.withColumn('loc0', when(lower(col("usr_location")).rlike("|".join(["(" + kw.lower() + ")" for kw in af_full1])),lit(4)).otherwise(lit(0)))\
                                .withColumn('loc1', when(col("usr_location").rlike("|".join(["(" + kw + ")" for kw in af_3])),lit(4)).otherwise(lit(0)))\
                                .withColumn('loc2', when(lower(col("usr_location")).rlike("|".join(["(" + kw.lower() + ")" for kw in af_full])),lit(3)).otherwise(lit(0)))\
                                .withColumn('loc3', when(col("usr_location").rlike("|".join(["(" + kw + ")" for kw in af_2])),lit(2)).otherwise(lit(0)))\
                                .withColumn('continent',lit('AF'))
    
tweets_af = tweets_af.withColumn("score", col('loc0')+col('loc1')+col('loc2')+col('loc3'))

tweets_af = tweets_af.select('usr_id','usr_location','score','continent')\
                     .filter(col('score').isNotNull()).orderBy('score',ascending=False)

In [16]:
tweets_eu = tweets_usr_locations.withColumn('loc0', when(lower(col("usr_location")).rlike("|".join(["(" + kw.lower() + ")" for kw in eu_full1])),lit(4)).otherwise(lit(0)))\
                                .withColumn('loc1', when(col("usr_location").rlike("|".join(["(" + kw + ")" for kw in eu_3])),lit(4)).otherwise(lit(0)))\
                                .withColumn('loc2', when(lower(col("usr_location")).rlike("|".join(["(" + kw.lower() + ")" for kw in eu_full])),lit(3)).otherwise(lit(0)))\
                                .withColumn('loc3', when(col("usr_location").rlike("|".join(["(" + kw + ")" for kw in eu_2])),lit(2)).otherwise(lit(0)))\
                                .withColumn('continent',lit('EU'))
    
tweets_eu = tweets_eu.withColumn("score", col('loc0')+col('loc1')+col('loc2')+col('loc3'))

tweets_eu = tweets_eu.select('usr_id','usr_location','score','continent')\
                     .filter(col('score').isNotNull()).orderBy('score',ascending=False)

In [17]:
tweets_sa = tweets_usr_locations.withColumn('loc0', when(lower(col("usr_location")).rlike("|".join(["(" + kw.lower() + ")" for kw in sa_full1])),lit(4)).otherwise(lit(0)))\
                                .withColumn('loc1', when(col("usr_location").rlike("|".join(["(" + kw + ")" for kw in sa_3])),lit(4)).otherwise(lit(0)))\
                                .withColumn('loc2', when(lower(col("usr_location")).rlike("|".join(["(" + kw.lower() + ")" for kw in sa_full])),lit(3)).otherwise(lit(0)))\
                                .withColumn('loc3', when(col("usr_location").rlike("|".join(["(" + kw + ")" for kw in sa_2])),lit(2)).otherwise(lit(0)))\
                                .withColumn('continent',lit('SA'))
    
tweets_sa = tweets_sa.withColumn("score", col('loc0')+col('loc1')+col('loc2')+col('loc3'))

tweets_sa = tweets_sa.select('usr_id','usr_location','score','continent')\
                     .filter(col('score').isNotNull()).orderBy('score',ascending=False)

In [18]:
tweets_oc = tweets_usr_locations.withColumn('loc0', when(lower(col("usr_location")).rlike("|".join(["(" + kw.lower() + ")" for kw in oc_full1])),lit(4)).otherwise(lit(0)))\
                                .withColumn('loc1', when(col("usr_location").rlike("|".join(["(" + kw + ")" for kw in oc_3])),lit(4)).otherwise(lit(0)))\
                                .withColumn('loc2', when(lower(col("usr_location")).rlike("|".join(["(" + kw.lower() + ")" for kw in oc_full])),lit(3)).otherwise(lit(0)))\
                                .withColumn('loc3', when(col("usr_location").rlike("|".join(["(" + kw + ")" for kw in oc_2])),lit(2)).otherwise(lit(0)))\
                                .withColumn('continent',lit('OC'))
    
tweets_oc = tweets_oc.withColumn("score", col('loc0')+col('loc1')+col('loc2')+col('loc3'))

tweets_oc = tweets_oc.select('usr_id','usr_location','score','continent')\
                     .filter(col('score').isNotNull()).orderBy('score',ascending=False)

In [19]:
tweets_loc = tweets_na.union(tweets_as)\
                      .union(tweets_af)\
                      .union(tweets_eu)\
                      .union(tweets_oc)\
                      .union(tweets_sa)

In [20]:
tweets_score1 = tweets_loc.orderBy(col('score'),ascending=False)\
                          .groupby(['usr_id','usr_location','score']).agg(sum(lit(1)).alias('freq'))

tweets_score1 = tweets_score1.filter(col('freq')==1).drop('freq')

In [21]:
w = Window.partitionBy("usr_id").orderBy(col("score").desc())

tweets_score2 = tweets_loc.withColumn("row",row_number().over(w))\
                          .filter(col("row") == 1).drop("row")\
                          .filter(col('score') > 0)

In [22]:
usr_continent = tweets_score1.join(tweets_score2, ["usr_id","usr_location","score"],"inner")\
                             .drop('score')

In [23]:
# %%time

# usr_continent.show(50,False)

In [24]:
tweets_df_cont = tweets_df.join(usr_continent, ['usr_id','usr_location'], 'left')

tweets_df_cont = tweets_df_cont.withColumn("twt_continent1", when(col("twt_country").rlike("|".join(["(" + kw + ")" for kw in na_2])),lit('NA')).\
                                                             when(col("twt_country").rlike("|".join(["(" + kw + ")" for kw in as_2])),lit('AS')).\
                                                             when(col("twt_country").rlike("|".join(["(" + kw + ")" for kw in af_2])),lit('AF')).\
                                                             when(col("twt_country").rlike("|".join(["(" + kw + ")" for kw in eu_2])),lit('EU')).\
                                                             when(col("twt_country").rlike("|".join(["(" + kw + ")" for kw in sa_2])),lit('SA')).\
                                                             when(col("twt_country").rlike("|".join(["(" + kw + ")" for kw in oc_2])),lit('OC')))

tweets_df_cont = tweets_df_cont.withColumn("twt_continent", when(col("twt_continent1").isNotNull(),col("twt_continent1")).otherwise(col("continent")))\
                               .drop('continent','twt_continent1','created_at','reply_count','twt_country','twt_location','twt_location_full','twt_location_type')

tweets_df_cont = tweets_df_cont.select(sorted(tweets_df_cont.columns))

In [ ]:
tweets_df_cont_o = tweets_df_cont.filter(col('is_original')==1)
tweets_df_cont_o.select([count(when(col(c).isNull(), c)).alias(c) for c in tweets_df_cont_o.columns]).show(vertical=True)

-RECORD 0------------------
 dt_date         | 0       
 dt_datehour     | 0       
 dt_day          | 0       
 dt_hour         | 0       
 dt_month        | 0       
 dt_year         | 0       
 ext_qt_user_id  | 2739456 
 ext_rp_user_id  | 2648070 
 ext_rt_id       | 2739456 
 ext_rt_user_id  | 2739456 
 is_original     | 0       
 twt_continent   | 1574108 
 twt_hashtags    | 0       
 twt_id          | 0       
 twt_importance  | 0       
 twt_likes       | 0       
 twt_quote_count | 0       
 twt_reply_count | 0       
 twt_retwt_count | 0       
 twt_text        | 0       
 usr_desc        | 278311  
 usr_followers   | 0       
 usr_id          | 0       
 usr_location    | 739986  
 usr_name        | 0       
 usr_tweet_count | 0       
 usr_verified    | 0       



In [ ]:
tweets_df_cont.select([count(when(col(c).isNull(), c)).alias(c) for c in tweets_df_cont.columns]).show(vertical=True)

-RECORD 0-------------------
 dt_date         | 0        
 dt_datehour     | 0        
 dt_day          | 0        
 dt_hour         | 0        
 dt_month        | 0        
 dt_year         | 0        
 ext_qt_user_id  | 19016605 
 ext_rp_user_id  | 20700808 
 ext_rt_id       | 7350818  
 ext_rt_user_id  | 7350818  
 is_original     | 0        
 twt_continent   | 15603656 
 twt_hashtags    | 0        
 twt_id          | 0        
 twt_importance  | 0        
 twt_likes       | 0        
 twt_quote_count | 0        
 twt_reply_count | 0        
 twt_retwt_count | 0        
 twt_text        | 0        
 usr_desc        | 4470803  
 usr_followers   | 0        
 usr_id          | 0        
 usr_location    | 9017922  
 usr_name        | 0        
 usr_tweet_count | 0        
 usr_verified    | 0        



In [ ]:
tweets_df_cont.count()

23913387

total rows = 23913387

total continent null = 15603656

In [ ]:
tweets_df_cont = tweets_df_cont.select(sorted(tweets_df_cont.columns))

In [ ]:
%%time

# Delete folder from COS bucket
def delete_folder(bucket_name, folder_name):
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))

    for blob in blobs:
        blob.delete()

delete_folder('msca-bdp-students-bucket', 'shared_data/kaihayden/final2')

CPU times: user 483 ms, sys: 9.66 ms, total: 492 ms
Wall time: 7.17 s


In [ ]:
%%time

tweets_df_cont.write.format("parquet")\
              .mode('overwrite')\
              .save('gs://msca-bdp-students-bucket/shared_data/kaihayden/final2')

CPU times: user 46.2 ms, sys: 20 ms, total: 66.2 ms
Wall time: 6min 6s


In [ ]:
# tweets_df_cont.columns

['dt_date',
 'dt_datehour',
 'dt_day',
 'dt_hour',
 'dt_month',
 'dt_year',
 'ext_qt_user_id',
 'ext_rp_user_id',
 'ext_rt_id',
 'ext_rt_user_id',
 'is_original',
 'twt_continent',
 'twt_hashtags',
 'twt_id',
 'twt_importance',
 'twt_likes',
 'twt_quote_count',
 'twt_reply_count',
 'twt_retwt_count',
 'twt_text',
 'usr_desc',
 'usr_followers',
 'usr_id',
 'usr_location',
 'usr_name',
 'usr_tweet_count',
 'usr_verified']